In [ ]:
import pandas_datareader.data as web
import datetime
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2021,9,1)
df = web.DataReader('GOOGL','stooq',start,end)

In [ ]:

def Stock_Price_LSTM_Date_Precesing(df,mem_his_days,pre_days):
    df.dropna(inplace=True)#删除空值
    df.sort_index(inplace=True)#对数据进行排序，inplace=True按照时间正序排列；

    df['label'] = df['Close'].shift(-pre_days)#定义label，让它与Close相差10天的值
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    sca_X = scaler.fit_transform(df.iloc[:,:-1])#对数据进行标准化
    #对特征向量空间进行设定

    from collections import deque
    deq = deque(maxlen=mem_his_days)#先进先出的队列
    X = []
    for i in sca_X:
        deq.append(list(i))
        if len(deq)==mem_his_days:
            X.append(list(deq))
    X_lately = X[-pre_days:]
    X = X[:-pre_days]
    y = df['label'].values[mem_his_days-1:-pre_days]
    import numpy as np
    X = np.array(X)
    y = np.array(y)
    return X,y,X_lately

In [ ]:
X,y,X_lately = Stock_Price_LSTM_Date_Precesing(df,5,10)

print(len(X))
print(len(y))
print(len(X_lately))

In [ ]:
#用循环来找最优的超参数       思考：换成grid search
pre_days = 10
# mem_days = [5,10,15]
# lstm_layers = [1,2,3]
# dense_layers = [1,2,3]
# units = [16,32]
mem_days = [5]
lstm_layers = [1]
dense_layers = [1]
units = [32]
from tensorflow.keras.callbacks import ModelCheckpoint
for the_mem_days in mem_days:
    for the_lstm_layers in lstm_layers:
        for the_dense_layers in dense_layers:
            for the_unints in units:
                filepath = './models/{val_mape:.2f}_{epoch:02d}_'+f'mem_{the_lstm_layers}_lstm_{the_lstm_layers}_dense_{the_dense_layers}_unit_{the_unints}'
                checkpoint = ModelCheckpoint(
                    filepath=filepath,
                    save_weights_only=False,
                    monitor='val_mape',
                    mode='min',
                    save_best_only=True)

                X,y,X_lately = Stock_Price_LSTM_Date_Precesing(df,the_mem_days,pre_days)
                from sklearn.model_selection import train_test_split#划分测试集与训练集
                X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.1)

                import tensorflow as tf
                from tensorflow.keras.models import Sequential
                from tensorflow.keras.layers import LSTM,Dense,Dropout
                model = Sequential()
                model.add(LSTM(the_unints,input_shape=X.shape[1:],activation='tanh',return_sequences=True))#第一层
                model.add(Dropout(0.1))

                for i in range(the_lstm_layers):
                    model.add(LSTM(the_unints,activation='tanh',return_sequences=True))#第二层
                    model.add(Dropout(0.1))
                
                
                model.add(LSTM(the_unints,activation='tanh'))#第三层
                model.add(Dropout(0.1))
                
                for i in range(the_dense_layers):
                    model.add(Dense(the_unints,activation='relu'))#构建一个全连接层
                    model.add(Dropout(0.1))

                model.add(Dense(1))
                model.compile(optimizer='adam',loss='mse',metrics=['mape'])
                model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test),callbacks=[checkpoint])#训练模型


In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/4.06_11_mem_1_lstm_1_dense_1_unit_32')

pre = best_model.predict(X_test)
print(pre)
print(len(pre))

In [ ]:
import matplotlib.pyplot as plt
df_time = df.index[-len(y_test):]
plt.plot(df_time,y_test,color='red',label='price')
plt.plot(df_time,pre,color='green',label='predict')
plt.show()